# Concurrency

A **concurrent** program is one consisting of multiple threads executing "at the same time" and independently.

A **thread** is an independent line of execution within a process.

A thread has its own set of registers (its context!) on the CPU.

A thread has its own stack.

A thread its own instruction pointer (aka program counter).

Within a process, the rest of the address space is shared amongst all of its threads:

- heap
- code block
- static global block


<br>
<img src="images/01-AS.png" width="500">
<br>

## Why talk about concurrency in OS?

The OS is the first concurrent program and needs to support concurrency in order to exist.

The mechanisms to support concurrency are critical to a computer operating safely and must be with the OS's purview.

